In [1]:
import piplite
await piplite.install(["resotonotebook==0.2.1", "plotly==5.10.0"])

# Resoto jupyter notebook

Welcome to Resoto!

This is a showcase of how you can interact with resoto using jupyter notebooks.

Try running the cells below and explore the capabilities.

After the installation, we need a few imports:

In [3]:
from resotonotebook import ResotoNotebook

The main interface is the `ResotoNotebook` class. It provides the methods to search the graph and visualize the results.

To create the `ResotoNotebook` object, we need to pass the pre shared key (PSK). If you started the resotocore without the `--psk` option (e.g. in your local setup), you can pass None as the PSK: 

In [4]:
rnb = ResotoNotebook("", psk=None)

Once we have the `ResotoNotebook` object, we can start performing searches.



### Counting

Get number of all collected instances by kind

In [ ]:
(await rnb.search("is(instance)")).groupby(["kind"])["kind"].count()

Using the cli command:

In [ ]:
await rnb.cli_execute("search is(instance) | count kind")

### Searching by kind

Get list of all the DigitalOcean droplets:

In [ ]:
await rnb.search("is(digitalocean_droplet)")

Using the cli command:

In [ ]:
await rnb.cli_execute("search is(digitalocean_droplet)")

### Visualization

Show the instances/cores/creation time heatmap:

In [ ]:
import plotly.express as px
data = await rnb.search("is(instance)")
px.density_heatmap(data, x="account_id", y="instance_cores") 

### Selecting properties

Get list of name, type, cores, and memory for each instance:

In [ ]:
(await rnb.search("is(instance)"))[["instance_type","instance_cores", "instance_memory"]]

Using the cli command:

In [ ]:
await rnb.cli_execute("search is(instance) | list instance_type, instance_cores as cores, instance_memory as memory, /ancestors.account.reported.name as account")

Get a list of instance IDs and their creation times:

In [ ]:
(await rnb.search("is(instance)"))[["id", "ctime"]]

Using the cli command:

In [ ]:
await rnb.cli_execute('search is(instance) | list id, ctime')

### Filtering

Get list of all compute instances with more than two CPU cores:

In [ ]:
result = await rnb.search("is(instance) and instance_cores > 2")
result['id']

Get list volumes that are not in use, larger than 10GB, older than 30 days. 


In [ ]:
(await rnb.search("is(volume) and volume_status != in-use and volume_size > 10 and age > 30d"))['id']

### Aggregation
Count the number of instances by account ID:

In [ ]:
result = await rnb.search("is(instance)")
result.groupby(["account_id"])["account_id"].count()

Aggregate RAM usage (bytes) data grouped by cloud, region, and instance type:

In [ ]:
result = await rnb.search("is(instance) and instance_status == running")
result \
    .groupby(["cloud_id", "region_id", "instance_type"], as_index=False)[["instance_cores"]] \
    .sum()

Using the cli:

In [ ]:
query = """
search is(instance) and instance_status == running | aggregate
  /ancestors.cloud.reported.name as cloud,
  /ancestors.region.reported.name as region,
  instance_type as type: sum(instance_memory) as memory_bytes
  """
await rnb.cli_execute(query)